In [ ]:
# Create database if not exists
spark.sql("CREATE DATABASE IF NOT EXISTS infra_intel_reporting")

# Step 1: Create a temporary view for switch interfaces
spark.sql(
    """
    CREATE OR REPLACE TEMPORARY VIEW switch_interfaces AS
    SELECT 
        lw.device_name, 
        DATE(lw.event_timestamp) AS event_date, 
        lw.device_type, 
        lw.region, 
        lw.location, 
        di.int_type, 
        di.int_last_change, 
        di.interface_desc, 
        di.description, 
        di.int_oper_status, 
        lw.host_server
    FROM edl.nmis9_device_events lw
    JOIN edl.nmis9_device_interface_events di ON lw.uuid = di.uuid
    WHERE lw.event_timestamp >= (current_timestamp() - INTERVAL 30 DAYS)
      AND lw.device_type = 'Switch'
      AND di.int_type = 'ethernetCsmacd'
      AND (di.int_oper_status = 'up' OR di.interface_desc != 'GigabitEthernet0/0')
      AND di.interface_desc NOT IN ('Bluetooth0/4', 'null')
      AND di.interface_desc NOT LIKE '%AppGigabitEthernet%'
      AND lw.location != 'Unknown'
    """
)

# Step 2: Create a temporary view for latest record per day with aggregation
spark.sql(
    """
    CREATE OR REPLACE TEMPORARY VIEW latest_record_per_day AS
    SELECT
        si.device_name,
        si.event_date,
        si.device_type,
        si.region,
        si.location,
        si.int_type,
        si.interface_desc,
        MAX(si.int_last_change) as int_last_change,
        MAX(si.description) as description,
        MAX(si.int_oper_status) as int_oper_status,
        si.host_server
    FROM switch_interfaces si
    GROUP BY
        si.device_name,
        si.event_date,
        si.device_type,
        si.region,
        si.location,
        si.int_type,
        si.interface_desc,
        si.host_server
    """
)

# Persist the intermediate result to avoid recomputation
latest_record_per_day_df = spark.sql("SELECT * FROM latest_record_per_day")
latest_record_per_day_df.persist()

# Step 3: Create a temporary view for status tracker
latest_record_per_day_df.createOrReplaceTempView("latest_record_per_day_persisted")
spark.sql(
    """
    CREATE OR REPLACE TEMPORARY VIEW status_tracker AS
    SELECT
        lr.device_name,
        lr.event_date,
        lr.device_type,
        lr.region,
        lr.location,
        lr.int_type,
        lr.interface_desc,
        lr.int_last_change,
        lr.description,
        lr.int_oper_status,
        lr.host_server,
        LAG(lr.int_oper_status) OVER (PARTITION BY lr.device_name, lr.interface_desc ORDER BY lr.event_date) as prev_int_oper_status
    FROM latest_record_per_day_persisted lr
    """
)

# Step 4: Create a temporary view for status groups
spark.sql(
    """
    CREATE OR REPLACE TEMPORARY VIEW status_groups AS
    SELECT
        st.device_name,
        st.event_date,
        st.device_type,
        st.region,
        st.location,
        st.int_type,
        st.interface_desc,
        st.int_last_change,
        st.description,
        st.int_oper_status,
        st.host_server,
        st.prev_int_oper_status,
        CASE 
            WHEN st.prev_int_oper_status = st.int_oper_status THEN 0
            ELSE 1
        END AS status_switch,
        SUM(CASE 
            WHEN st.prev_int_oper_status = st.int_oper_status THEN 0
            ELSE 1
        END) OVER (PARTITION BY st.device_name, st.interface_desc ORDER BY st.event_date) AS status_group
    FROM status_tracker st
    """
)

# Step 5: Create a temporary view for consecutive days
spark.sql(
    """
    CREATE OR REPLACE TEMPORARY VIEW consecutive_days AS
    SELECT
        sg.device_name, 
        sg.event_date, 
        sg.device_type, 
        sg.region, 
        sg.location, 
        sg.int_type, 
        sg.interface_desc,
        sg.int_last_change,
        sg.description,
        sg.host_server,
        sg.int_oper_status,
        MIN(sg.event_date) OVER (PARTITION BY sg.device_name, sg.interface_desc, sg.status_group) AS min_event_date,
        COUNT(*) OVER (PARTITION BY sg.device_name, sg.interface_desc, sg.status_group ORDER BY sg.event_date) AS consecutive_days
    FROM status_groups sg
    """
)

# Persist the intermediate result to avoid recomputation
consecutive_days_df = spark.sql("SELECT * FROM consecutive_days")
consecutive_days_df.persist()

# Step 6: Create the final DataFrame
consecutive_days_df.createOrReplaceTempView("consecutive_days_persisted")
df = spark.sql(
    """
    SELECT
        cd.device_name, 
        cd.event_date, 
        cd.device_type, 
        cd.region, 
        cd.location, 
        cd.int_type, 
        cd.interface_desc,
        cd.int_last_change,
        cd.description,
        cd.host_server,
        cd.int_oper_status,
        CASE 
            WHEN cd.consecutive_days < 30 THEN cd.consecutive_days
            ELSE 30
        END AS filtered_consecutive_days,
        CASE 
            WHEN cd.int_oper_status = 'down' AND 
                  CAST(REGEXP_EXTRACT(cd.int_last_change, '([0-9]+) days') AS INT) >= 90 
                  THEN CAST(REGEXP_EXTRACT(cd.int_last_change, '([0-9]+) days') AS STRING) 
            ELSE NULL 
        END AS `90+ days`
    FROM consecutive_days_persisted cd
    ORDER BY cd.event_date DESC
    """
)

# Write the DataFrame to the specified path and save as a table
df.write.option(
    "path", "/mnt/sandbox/AWS-EDL-INFRA-INTEL-DATA/reporting/nmis_interface_metrics"
).saveAsTable(
    "infra_intel_reporting.nmis_interface_metrics",
    format="parquet",
    mode="overwrite",
)

In [ ]:
spark.sql(
    """
CREATE OR REPLACE VIEW edl_views.nmis9_report
TBLPROPERTIES (
    "edl_sources" = "com.deere.enterprise.datalake.enhance.iit_reporting") AS
select * from infra_intel_reporting.nmis_interface_metrics
"""
)